# Exploration

Exploratory work for solving the problem of (1) a more efficient algorithm for discovering dependencies that (2) handles the cases of "ties," or dependencies that are invarient to one another.

## Let's start with (1), the problem with the existing approach

Here's an example:

In [8]:
# Example dictionary of items we want to resolve dependencies for
my_items = {
    'A': ['B', 'C', 'D'],  # -- A is dependent on B, C, D,
    'B': [],  # -- B is dependent on nothing, etc.
    'C': ['D'],
    'D': ['B', 'E'],
    'E': ['F'],
    'F': [],
    'Z': ['A', 'B', 'C', 'D']
}


def dependencies_exist(my_items):
    all_dependencies_exist = True
    for item, dependencies in my_items.items():
        for dependency in dependencies:
            if dependency not in my_items.keys():
                print('Non-existant dependency: ({0}, {1})').format(
                    item, dependency)
                all_dependencies_exist = False
    return all_dependencies_exist


def list_dependencies(my_items, item, verbose=False):
    """Find all dependencies for [item] within [my_items].
    """
    dependencies, new_dependencies_count = list(my_items[item]), -1
    if verbose:
        print("> Indexing:", item)
    num_indent = 2
    while new_dependencies_count != 0:
        new_dependencies_count = 0
        for item in dependencies:
            new_dependencies = [new_item for new_item in my_items[item] 
                                if new_item not in dependencies]
            if verbose:
                print("{} Indexing:".format(">" * num_indent), item)
            dependencies += new_dependencies
            new_dependencies_count += len(new_dependencies)
        num_indent += 1
    return dependencies


def no_circular_dependencies(my_items):
    """Check for any circular dependencies in [my_items].
    """
    not_circular = True
    for item in my_items.keys():
        dependencies = list_dependencies(my_items, item)
        if item in dependencies:
            not_circular = False
            print('Circular dependency for', item)
    return not_circular



print(dependencies_exist(my_items))
print(no_circular_dependencies(my_items))

True
True


The problem is that we're repeating many of the computations many, many times!

In [13]:
def order_dependencies(my_items, verbose=False):
    """Order the dependencies in [my_items], returning a list of keys from
    [my_items] in order such that all dependencies resolve.
    """
    items, index = list(my_items.keys()), 0
    while (len(items) - 1) != index:
        item = items[index]
        if verbose:
            print("Identifying order for:", item)
        current_dependencies = list_dependencies(my_items, item, verbose=verbose)
        no_order_change = True
        for dependency in current_dependencies:
            if items.index(dependency) - items.index(item) > 0:
                items += [items.pop(index)]
                no_order_change = False
                break
        if no_order_change:
            index += 1
    return items


order_dependencies(my_items, verbose=True)

Identifying order for: A
> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>> Indexing: E
>> Indexing: F
>>> Indexing: B
>>> Indexing: C
>>> Indexing: D
>>> Indexing: E
>>> Indexing: F
Identifying order for: B
> Indexing: B
Identifying order for: C
> Indexing: C
>> Indexing: D
>> Indexing: B
>> Indexing: E
>> Indexing: F
>>> Indexing: D
>>> Indexing: B
>>> Indexing: E
>>> Indexing: F
Identifying order for: D
> Indexing: D
>> Indexing: B
>> Indexing: E
>> Indexing: F
>>> Indexing: B
>>> Indexing: E
>>> Indexing: F
Identifying order for: E
> Indexing: E
>> Indexing: F
Identifying order for: F
> Indexing: F
Identifying order for: Z
> Indexing: Z
>> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>> Indexing: E
>> Indexing: F
>>> Indexing: A
>>> Indexing: B
>>> Indexing: C
>>> Indexing: D
>>> Indexing: E
>>> Indexing: F
Identifying order for: A
> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>> Indexing: E
>> Indexing: F
>>> Indexing: B
>>> Indexing: C
>>> Inde

['B', 'F', 'E', 'D', 'C', 'A', 'Z']

Here's a clearer example, where the problem is truly highlighted:

In [21]:
my_items = {
    "A": ["B"],
    "B": ["C"],
    "C": ["D"],
    "D": []
}


order_dependencies(my_items, verbose=True)

Identifying order for: A
> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: B
>>> Indexing: C
>>> Indexing: D
Identifying order for: B
> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: C
>>> Indexing: D
Identifying order for: C
> Indexing: C
>> Indexing: D
Identifying order for: D
> Indexing: D
Identifying order for: A
> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: B
>>> Indexing: C
>>> Indexing: D
Identifying order for: B
> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: C
>>> Indexing: D
Identifying order for: C
> Indexing: C
>> Indexing: D
Identifying order for: A
> Indexing: A
>> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: B
>>> Indexing: C
>>> Indexing: D
Identifying order for: B
> Indexing: B
>> Indexing: C
>> Indexing: D
>>> Indexing: C
>>> Indexing: D


['D', 'C', 'B', 'A']

### Solution for (1) using recursion

The process for just listing the dependencies should also give us the order, these don't need to be separate steps, perhaps except for getting the ideal ordering.

In [78]:
def enhanced_list_dependencies(my_items, item, known_dependencies={}, debug=False):
    """Better version of list_dependencies
    """
    
    # List of dependnecies for an item, initially populate with the known dependencies
    item_dependencies = my_items[item]
    if not isinstance(item_dependencies, list):
        item_dependencies = list(item_dependencies)
    if debug:
        print("Initial item dependencies for {}: ".format(item), item_dependencies)
        print("Current state of known dependencies:", known_dependencies)
    
    # Traverse the tree of dependencies
    for dependency in item_dependencies:
        if debug:
            print("> Looking into dependency: ", dependency)
        try:
            # We've already cached the dependency
            item_dependencies.append(list(known_dependencies[dependency]))
            if debug:
                print(">> Dependency for {} known: ".format(dependency), known_dependencies[dependency])
        except KeyError:
            # Recursively call this function until we known all of the possible dependencies
            if debug:
                print(">> Dependency for {} unknown, initiating recursive call ..... ".format(dependency))
            newly_discovered_dependencies, known_dependencies = enhanced_list_dependencies(
                my_items, dependency, known_dependencies, debug) 
                
            # Get rid of empty lists in the output, combine the new and existing dependencies
            try:
                item_dependencies = [existing_dependency for existing_dependency 
                                     in item_dependencies if existing_dependency]
                newly_discovered_dependencies = [new_discovery for new_discovery 
                                                 in newly_discovered_dependencies if new_discovery]
                item_dependencies = list(set(item_dependencies + newly_discovered_dependencies))  # -- old + new
            except TypeError:
                print("This is all part of finding deps for", item)
                print("TYPeERROR: new dependencies: ", newly_discovered_dependencies)
                print("TYPeERROR: existing dependencies: ", item_dependencies)
            
    # When we have all of an item's dependencies, add them to the cache, and return the list of
    # the item's dependencies
    if debug:
        print("& All dependencies for {} are known :) they are: ".format(item), item_dependencies)
    known_dependencies[item] = item_dependencies
    return item_dependencies, known_dependencies
            
      

my_items = {
    "A": ["B"],
    "B": ["C"],
    "C": ["D"],
    "D": []
}
        
        
enhanced_list_dependencies(my_items, "D", known_dependencies={}, debug=True)

Initial item dependencies for D:  []
Current state of known dependencies: {}
& All dependencies for D are known :) they are:  []


([], {'D': []})

In [85]:
def enhanced_list_dependencies(my_items, item, known_dependencies={}, debug=False):
    """Better version of list_dependencies
    """
    
    # List of dependnecies for an item, initially populate with the known dependencies
    item_dependencies = my_items[item]
    if not isinstance(item_dependencies, list):
        item_dependencies = list(item_dependencies)
    if debug:
        print("Initial item dependencies for {}: ".format(item), item_dependencies)
        print("Current state of known dependencies:", known_dependencies)
    
    # Traverse the tree of dependencies
    for dependency in item_dependencies:
        if debug:
            print("> Looking into dependency: ", dependency)
        try:
            # We've already cached the dependency
            cached_dependencies = known_dependencies[dependency]
            if not isinstance(cached_dependencies, list):
                cached_dependencies = list(cached_dependencies)
            if cached_dependencies:
                item_dependencies = list(set(item_dependencies + cached_dependencies))  # -- old + new
            if debug:
                print(">> Dependency for {} known: ".format(dependency), known_dependencies[dependency])
        except KeyError:
            # Recursively call this function until we known all of the possible dependencies
            if debug:
                print(">> Dependency for {} unknown, initiating recursive call ..... ".format(dependency))
            newly_discovered_dependencies, known_dependencies = enhanced_list_dependencies(
                my_items, dependency, known_dependencies, debug) 
            if newly_discovered_dependencies:
                item_dependencies = list(set(item_dependencies + newly_discovered_dependencies))  # -- old + new
    
    # When we have all of an item's dependencies, add them to the cache, and return the list of
    # the item's dependencies
    if debug:
        print("& All dependencies for {} are known :) they are: ".format(item), item_dependencies)
    known_dependencies[item] = item_dependencies
    return item_dependencies, known_dependencies


my_items = {
    'A': ['B', 'C', 'D'],  # -- A is dependent on B, C, D,
    'B': [],  # -- B is dependent on nothing, etc.
    'C': ['D'],
    'D': ['B', 'E'],
    'E': ['F'],
    'F': [],
    'Z': ['A', 'B', 'C', 'D']
}

enhanced_list_dependencies(my_items, "Z", known_dependencies={}, debug=True)

Initial item dependencies for Z:  ['A', 'B', 'C', 'D']
Current state of known dependencies: {}
> Looking into dependency:  A
>> Dependency for A unknown, initiating recursive call ..... 
Initial item dependencies for A:  ['B', 'C', 'D']
Current state of known dependencies: {}
> Looking into dependency:  B
>> Dependency for B unknown, initiating recursive call ..... 
Initial item dependencies for B:  []
Current state of known dependencies: {}
& All dependencies for B are known :) they are:  []
> Looking into dependency:  C
>> Dependency for C unknown, initiating recursive call ..... 
Initial item dependencies for C:  ['D']
Current state of known dependencies: {'B': []}
> Looking into dependency:  D
>> Dependency for D unknown, initiating recursive call ..... 
Initial item dependencies for D:  ['B', 'E']
Current state of known dependencies: {'B': []}
> Looking into dependency:  B
>> Dependency for B known:  []
> Looking into dependency:  E
>> Dependency for E unknown, initiating recursiv

(['A', 'E', 'B', 'C', 'D', 'F'],
 {'B': [],
  'F': [],
  'E': ['F'],
  'D': ['B', 'F', 'E'],
  'C': ['B', 'F', 'D', 'E'],
  'A': ['E', 'B', 'C', 'D', 'F'],
  'Z': ['A', 'E', 'B', 'C', 'D', 'F']})

In [86]:
order_dependencies(my_items)

['B', 'F', 'E', 'D', 'C', 'A', 'Z']

In [88]:
my_items = {
    "A": ["B"],
    "B": ["C"],
    "C": ["D"],
    "D": []
}
        
        
enhanced_list_dependencies(my_items, "A", known_dependencies={}, debug=True)

Initial item dependencies for A:  ['B']
Current state of known dependencies: {}
> Looking into dependency:  B
>> Dependency for B unknown, initiating recursive call ..... 
Initial item dependencies for B:  ['C']
Current state of known dependencies: {}
> Looking into dependency:  C
>> Dependency for C unknown, initiating recursive call ..... 
Initial item dependencies for C:  ['D']
Current state of known dependencies: {}
> Looking into dependency:  D
>> Dependency for D unknown, initiating recursive call ..... 
Initial item dependencies for D:  []
Current state of known dependencies: {}
& All dependencies for D are known :) they are:  []
& All dependencies for C are known :) they are:  ['D']
& All dependencies for B are known :) they are:  ['C', 'D']
& All dependencies for A are known :) they are:  ['B', 'C', 'D']


(['B', 'C', 'D'], {'D': [], 'C': ['D'], 'B': ['C', 'D'], 'A': ['B', 'C', 'D']})

In [89]:
order_dependencies(my_items)

['D', 'C', 'B', 'A']

Notice that the order of keys in the `known_dependencies` dict returned by `enhanced_list_dependencies` is giving us the same order as `order_dependencies` :)

--------------------

Now, let's try saving `known_dependencies` and looping through all of the keys in `my_items` to find the dependencies for each, starting with `B`

In [91]:
my_items = {
    'A': ['B', 'C', 'D'],  # -- A is dependent on B, C, D,
    'B': [],  # -- B is dependent on nothing, etc.
    'C': ['D'],
    'D': ['B', 'E'],
    'E': ['F'],
    'F': [],
    'Z': ['A', 'B', 'C', 'D']
}

deps, known_dependencies = enhanced_list_dependencies(
    my_items, "B", known_dependencies={}, debug=True)

Initial item dependencies for B:  []
Current state of known dependencies: {}
& All dependencies for B are known :) they are:  []


Now let's get `D`, which should already know the result of `B`

In [92]:
deps, known_dependencies = enhanced_list_dependencies(
    my_items, "D", known_dependencies=known_dependencies, debug=True)

Initial item dependencies for D:  ['B', 'E']
Current state of known dependencies: {'B': []}
> Looking into dependency:  B
>> Dependency for B known:  []
> Looking into dependency:  E
>> Dependency for E unknown, initiating recursive call ..... 
Initial item dependencies for E:  ['F']
Current state of known dependencies: {'B': []}
> Looking into dependency:  F
>> Dependency for F unknown, initiating recursive call ..... 
Initial item dependencies for F:  []
Current state of known dependencies: {'B': []}
& All dependencies for F are known :) they are:  []
& All dependencies for E are known :) they are:  ['F']
& All dependencies for D are known :) they are:  ['B', 'F', 'E']


And `E` and `F`, which should already be known

In [93]:
deps, known_dependencies = enhanced_list_dependencies(
    my_items, "E", known_dependencies=known_dependencies, debug=True)

Initial item dependencies for E:  ['F']
Current state of known dependencies: {'B': [], 'F': [], 'E': ['F'], 'D': ['B', 'F', 'E']}
> Looking into dependency:  F
>> Dependency for F known:  []
& All dependencies for E are known :) they are:  ['F']


In [94]:
deps, known_dependencies = enhanced_list_dependencies(
    my_items, "F", known_dependencies=known_dependencies, debug=True)

Initial item dependencies for F:  []
Current state of known dependencies: {'B': [], 'F': [], 'E': ['F'], 'D': ['B', 'F', 'E']}
& All dependencies for F are known :) they are:  []


And `C`, which should only call `D` which is already known

In [95]:
deps, known_dependencies = enhanced_list_dependencies(
    my_items, "C", known_dependencies=known_dependencies, debug=True)

Initial item dependencies for C:  ['D']
Current state of known dependencies: {'B': [], 'F': [], 'E': ['F'], 'D': ['B', 'F', 'E']}
> Looking into dependency:  D
>> Dependency for D known:  ['B', 'F', 'E']
& All dependencies for C are known :) they are:  ['B', 'F', 'D', 'E']


And `A`, which should call `B`, `C`, and `D` which are all already known

In [96]:
deps, known_dependencies = enhanced_list_dependencies(
    my_items, "A", known_dependencies=known_dependencies, debug=True)

Initial item dependencies for A:  ['B', 'C', 'D']
Current state of known dependencies: {'B': [], 'F': [], 'E': ['F'], 'D': ['B', 'F', 'E'], 'C': ['B', 'F', 'D', 'E']}
> Looking into dependency:  B
>> Dependency for B known:  []
> Looking into dependency:  C
>> Dependency for C known:  ['B', 'F', 'D', 'E']
> Looking into dependency:  D
>> Dependency for D known:  ['B', 'F', 'E']
& All dependencies for A are known :) they are:  ['E', 'B', 'C', 'D', 'F']


And, finally, `Z`

In [97]:
deps, known_dependencies = enhanced_list_dependencies(
    my_items, "Z", known_dependencies=known_dependencies, debug=True)

Initial item dependencies for Z:  ['A', 'B', 'C', 'D']
Current state of known dependencies: {'B': [], 'F': [], 'E': ['F'], 'D': ['B', 'F', 'E'], 'C': ['B', 'F', 'D', 'E'], 'A': ['E', 'B', 'C', 'D', 'F']}
> Looking into dependency:  A
>> Dependency for A known:  ['E', 'B', 'C', 'D', 'F']
> Looking into dependency:  B
>> Dependency for B known:  []
> Looking into dependency:  C
>> Dependency for C known:  ['B', 'F', 'D', 'E']
> Looking into dependency:  D
>> Dependency for D known:  ['B', 'F', 'E']
& All dependencies for Z are known :) they are:  ['A', 'E', 'B', 'C', 'D', 'F']


One last check for now...

In [120]:
%%time

def extreme_dependency_generator(dependency_depth):
    """Generate a list of items where each item is dependent on the subsequent item, aka an 
    extreme case of dependencies.
    """
    items_with_dependencies = {}
    for number in range(1, dependency_depth):
        items_with_dependencies[number] = [number + 1]
    items_with_dependencies[dependency_depth] = []
    return items_with_dependencies


results = enhanced_list_dependencies(extreme_dependency_generator(100), 1, known_dependencies={}, debug=False)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 333 µs


How does the running time for the recursion depth of 100 compare with the old algorithm?

In [122]:
%%time

results = enhanced_list_dependencies(extreme_dependency_generator(999), 1, known_dependencies={}, debug=False)

CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 21 ms


In [135]:
%%time

results = enhanced_list_dependencies(extreme_dependency_generator(5000), 1, known_dependencies={}, debug=False)

RecursionError: maximum recursion depth exceeded in comparison

This algorithm is looking good...but how can we make it iterative instead of recursive? Personally, I like the elegance of the recursive solution, and it makes most sense to me, but Python isn't optimized for tail call recursion, and has a default recursion depth of 1,000, which is fairly low.

### Solution for (1) using iteration

My original solution used iteration, but in a bad way because computations were repeated (imagine computing the Fibonacci sequence where each step has to be recomputed for each subsequent step...not good).

In [ ]:
def enhanced_list_dependencies(my_items, item, known_dependencies={}, debug=False):
    """Better version of list_dependencies
    """
    
    # List of dependnecies for an item, initially populate with the known dependencies
    item_dependencies = my_items[item]
    if not isinstance(item_dependencies, list):
        item_dependencies = list(item_dependencies)
    if debug:
        print("Initial item dependencies for {}: ".format(item), item_dependencies)
        print("Current state of known dependencies:", known_dependencies)
    
    # Traverse the tree of dependencies
    for dependency in item_dependencies:
        if debug:
            print("> Looking into dependency: ", dependency)
        try:
            # We've already cached the dependency
            cached_dependencies = known_dependencies[dependency]
            if not isinstance(cached_dependencies, list):
                cached_dependencies = list(cached_dependencies)
            if cached_dependencies:
                item_dependencies = list(set(item_dependencies + cached_dependencies))  # -- old + new
            if debug:
                print(">> Dependency for {} known: ".format(dependency), known_dependencies[dependency])
        except KeyError:
            # Recursively call this function until we known all of the possible dependencies
            if debug:
                print(">> Dependency for {} unknown, initiating recursive call ..... ".format(dependency))
            newly_discovered_dependencies, known_dependencies = enhanced_list_dependencies(
                my_items, dependency, known_dependencies, debug) 
            if newly_discovered_dependencies:
                item_dependencies = list(set(item_dependencies + newly_discovered_dependencies))  # -- old + new
    
    # When we have all of an item's dependencies, add them to the cache, and return the list of
    # the item's dependencies
    if debug:
        print("& All dependencies for {} are known :) they are: ".format(item), item_dependencies)
    known_dependencies[item] = item_dependencies
    return item_dependencies, known_dependencies

In [ ]:
def enhanced_list_dependencies_iteration(my_items, item, known_dependencies={}, debug=False):
    """Better version of list_dependencies using iteration
    """
    
    # List of dependnecies for an item, initially populate with the known dependencies
    item_dependencies = my_items[item]
    if not isinstance(item_dependencies, list):
        item_dependencies = list(item_dependencies)
    if debug:
        print("Initial item dependencies for {}: ".format(item), item_dependencies)
        print("Current state of known dependencies:", known_dependencies)
        
    new_dependencies = True
    while new_dependencies:
        new_dependencies_count = False
        
        for dependency in item_dependencies:
            if debug:
                print("> Looking into dependency: ", dependency)
            try:
                # We've already cached the dependency
                cached_dependencies = known_dependencies[dependency]
                if not isinstance(cached_dependencies, list):
                    cached_dependencies = list(cached_dependencies)
                if cached_dependencies:
                    item_dependencies = list(set(item_dependencies + cached_dependencies))  # -- old + new
                if debug:
                    print(">> Dependency for {} known: ".format(dependency), known_dependencies[dependency])
            except KeyError:
                # Recursively call this function until we known all of the possible dependencies
                if debug:
                    print(">> Dependency for {} unknown, initiating recursive call ..... ".format(dependency))
                newly_discovered_dependencies, known_dependencies = enhanced_list_dependencies(
                    my_items, dependency, known_dependencies, debug) 
                if newly_discovered_dependencies:
                    item_dependencies = list(set(item_dependencies + newly_discovered_dependencies))  # -- old + new
                    
                new_dependencies_count = True
        

In [ ]:
def list_dependencies(my_items, item, verbose=False):
    """Find all dependencies for [item] within [my_items].
    """
    dependencies, new_dependencies_count = list(my_items[item]), -1
    if verbose:
        print("> Indexing:", item)
    num_indent = 2
    while new_dependencies_count != 0:
        new_dependencies_count = 0
        for item in dependencies:
            new_dependencies = [new_item for new_item in my_items[item] 
                                if new_item not in dependencies]
            if verbose:
                print("{} Indexing:".format(">" * num_indent), item)
            dependencies += new_dependencies
            new_dependencies_count += len(new_dependencies)
        num_indent += 1
    return dependencies